## Autocorrect and Spelling Checker
#### Batsal Ghimire

### Introduction
Spelling checker is a program that identifies when a word is misspelled and provides the closest suggestion from the initialized dictionary. Spelling checkers are used in many applications as an integrated feature, most commonly seen in search engines, word processors, email clients, etc. It's also common to see spelling checkers in large databases to make it easier to find the correct entries. Spelling check feature is a must for most modern apps since it makes the user workflow much faster and satisfying. However, many large corporations use spell checkers that are very complex and are trained using large amounts of data, which might not be available for a regular user. So, we will be implementing a version of a spell checker that is functional and understandable to a wide range of people.

### Goals
* To have a spelling checker that takes in misspelled words or sentences as the input and gives out the corrected version of it as an output.
* Has a reasonable and acceptable time complexity, memory complexity and accuracy.
* Uses the probability of the occurrence of the word to choose between conflicting choices.
* Can at least process the dictionary with a few thousand words in a reasonable time.



### Approaches to solving the problem
* First, we will read all the correct words that are contained in the initially given dictionary.
* Then, we will compare the input words to the known list of correct words.
* Finally, we will find the closest word to the inputted word from the dictionary and return it to the user.

This is the higher-level approach to solve this problem and will remain consistent throughout all detailed approaches. However, first we need to appreciate the challenges with this problem and understand that these algorithms will not provide the perfect result everytime.
For example: if we misspell the word 'work' as 'worke', there will be no way to distinguish as to whether the word is 'worked' or 'work'. To solve this, we will use probability of occurrence of the words, that we will discuss later.

### Implementations
In this project, I will be looking at two main implementations, and among these two main categories, I will also explore further improvements and changes that can be made. Furthermore, I will not begin with a brute force approach to the problem i.e. compare each letter of the inputted word, to each of the  words in the dictionary. This will substantially increase the time complexity and with the dictionary size I will be using, this is simply not feasible.

In [1]:
from __future__ import division

f = open("tyo.txt","r") #This is a text extracted from Gutenberg which will act as a test case.
text = f.read() #This will read the given text file.
print (len(text))

426016


In [2]:
g = open("wordlist.txt","r") #This is a second test file consisting of around 75000 English words.
texts = g.read() #Reads the text file.
print (len(texts))

75880


After reading the files, we will need to filter out the strings. Why? Firstly, in Python, the capitalized and small versions of the same letter are not considered to be equal. Moreover, since we will constantly be comparing many strings together, filtering the words will make out job much more manageable. We will also remove any numbers since autocorrecting the number is not possible. After this, we will have a list of words that consists of 26 small English alphabets.

I will be using a string manipulation package in this case to make the input more flexible. And, since I will be using the same filtering function for both the dictionary and the misspelled words, there might be numbers involved in it. I could have used the $txt.lower()$ to get lowercase letters and stripped the spaces with $txt.strip()$, but to make things more efficient, I will be using the 're' package that has certain built-in capabilities I will be making use of. 

In [3]:
import re
def filtered_words(text):
    return re.findall('[a-z]+',text.lower()) #Returns the list of lowercase words from the given text. Removes all numbers and spaces as well.

I will be justifying the use of a few modules that I will be using. The mmh3 will enable us to create hash functions very quickly. It takes in two arguments i.e. the element that is to be hashed and the seeds. The random module will be used to generate pseudo-random seeds.

In [4]:
#First I will import all the required packages.
import math
import random
import string
import mmh3
import hashlib
import numpy as np

### Counting Bloom Filters
Counting Bloom Filters are a special type of a data structure that can tell us whether an item is present in the given set or not. This data structure has an excellent constant time complexity as $O(k)$, where 'k' is the number of hash functions. They are also very memory efficient since only minimal data is stored. The price we pay for these efficiencies is that the data structure is probabilistic and there is a risk of getting a false positive. However, we can alter the value of the false positive rate by changing the memory size that is allocated. The optimal number of hash functions is necessary to prevent unnecessary overlapping or spare filling of the memory size. So, we give it as a function of the number of items and the memory size:
$$ k = \frac{m}{n} ln(2)$$
Depending on the number of items to be stored and an acceptable false positive rate, we can select the appropriate memory size given as:
$$ m = -\frac{n \times ln(P)}{(ln(2))^2}$$
Here, 'P' is the false positive probability. As we decrease the probability, we can see that memory size increases. 

### What we want from counting bloom filters?
The reason we use counting bloom filters is to see if each of the words that is inputted is in the initialized dictionary. In other words, we are checking if the word has been correctly spelled. If the word is correctly spelled, no more steps needs to be taken and we can simply return the word without any modifications. In our implementation, it will return $True$. But, if the word is not found, this guarantees that the word is misspelled because false negatives are not seen in counting bloom filters. So, we return $False$ and further processing occurs.

In [5]:
class CountingBloomFilter(object):
    #We pass in the false positive rate and the number of items that needs be stored as the parameter. This is because the optimal number of hash functions and the array size can be computed by using the formula if we  have these parameters.
    def __init__(self, fpr, num_item, seed, threshold):
        self.threshold = threshold
        self.seed = seed
        self.num_item = num_item
        self.fpr = fpr #I'm assuming that  fpr is given as a probability. If not, we can easily convert percentage into probability.
        
        #We will use the formula that we found above to find the size of the array and the number of hash functions.
        #First for memory_size, because we need it to find the num_hashfn.
        #Formula: m = -n ln(P)/(ln2)^2
        self.memory_size = abs(int(round(-self.num_item * math.log(self.fpr))/(math.log(2)**2))) #For this we need the math library
    
        #Now, for number of hash functions
        #Formula: k = (m/n)ln(2)
        self.num_hashfn = abs(int(round((self.memory_size/self.num_item)* math.log(2))))
        
        #We have the size of the list, so we can create the list and initialize it with all zeros.
        self.slots = self.memory_size * [0]
        
    def hash_cbf(self, item):
        #We use the mmh3 hash to get the hash function. mmh3 has two arguments, the first is the item which is fixed, but we differe the seed.
        for i in self.seed[:self.num_hashfn]:
            self.hash_val = mmh3.hash(item, i) % self.memory_size
        return self
    
    def search(self, item):
        #Seed should give us the same values as long as the initial state are the same.
        #We check if the slot value is greater than the threshold. For example: if a slot has value '2', then it compares it with the threshold we have set and return true if it is greater.
        for seed_val in self.seed[:self.num_hashfn]:
            if (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) > self.threshold: #For this Assignment, I have set all the threshold to be 0.
                pass
            else:
                return False
        return True
    
    def insert(self, item):
        for seed_val in self.seed[:self.num_hashfn]:
            #We get the index from the hash function, and we increase that position's value by 1.
            self.slots[mmh3.hash(item, seed_val)%self.memory_size] = (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) + 1
        return self
    
    def delete(self, item):
        #Searches if the item exists. If not, there is no way to delete it.
        if self.search(item) == True:
            for seed_val in self.seed[:self.num_hashfn]:
                #Similar to when inserting, the value is incremented by 1. While deleting, we decrease this value by 1.
                self.slots[mmh3.hash(item, seed_val)%self.memory_size] = (self.slots[mmh3.hash(item, seed_val)%self.memory_size]) - 1
        return self

### Longest Common Subsequence (LCS)
Our first implementation consists the use of the longest common subsequence. We will compare the longest common subsequence of the misspelled word and each of the words in our dictionary. The combination with the longest common subsequence will be the correct word. However, there are a few flaws with this assumption. Firstly, when we have thousands of words, there are multiple combinations which have the same longest common subsequence. So, our goal must be to reduce the number of combinations that give the same LCS. 

For this, a few assumptions can be made. First is that the first letter of the misspelled word is always correct. We are not bothered with other letters of the word, but the first letter is always correct. This will be one filter which will help increase accuracy.

Since we know that this problem satisfies both the properties required to be solved by dynamic programming, we can use dynamic programming to improve the efficiency.
1. Optimal Substructure: We can break down the problem into smaller parts that are subproblems by themselves. We can do this until the solution becomes trivial.
2. Overlapping subproblems: The solutions to the subproblems can be reused to solve other problems. We are using a LCS matrix to store these values so they don't have to be constantly recomputed.

In [6]:
def longest_common_subsequence(x, y): #Takes in the two strings to find the LCS of
    #Lengths of the two strings
    m = len(x)
    n = len(y)
    
    #Initialize the tables of size mxn with all zeros.
    tab = np.zeros((m, n))
    
    #If both the strings are null, return zero.
    if m == 0 or n == 0: 
        return 0
    
    #If the first character of the strings match (Base Case)
    if x[0] == y[0]:
        tab[0, 0] = 1
    
    #1st column: Helps us find the character in x that matches the first one in y.
    for i in range(1, m):
        if tab[i - 1, 0]:
            tab[i, 0] = 1
        else:
            if x[i] == y[0]:
                tab[i, 0] = 1
                
    #1st row: Helps us find the character in y that matches the first one in x.
    for j in range(1, n):
        if tab[0, j - 1]:
            tab[0, j] = 1
        else:
            if x[0] == y[j]:
                tab[0, j] = 1
    
    #Apply the optimal substructure of the LCS problem.
    for i in range(1, m): #Each character in string 1
        for j in range(1, n): #Each character in string 2
            #If the two characters match, set the diagonals to +1
            if x[i] == y[j]:
                tab[i, j] = tab[i - 1, j - 1] + 1
            #If the two characters match, set the upper ones to +1
            elif tab[i - 1, j] >= tab[i, j - 1]:
                tab[i, j] = tab[i - 1, j]
            else: 
                tab[i, j] = tab[i, j - 1]
    
    return tab[-1,-1] #Return the last value from the table i.e. the longest common subsequence

In this step, we will apply another filter that can help us improve even further upon the accuracy. This was done because words like 'thesee' were being corrected as 'tennessee', where the difference between the length of the two words are more than with 'these'. So, we will only consider those words that are within a certain range of the initial length of the word.

In [7]:
def corrected_word(search_word, correct_words, thres): #This function returns the corrected word
    checker_text = filtered_words(correct_words) #Filters the words from the dictionary.
    typo_word = filtered_words(search_word) #This will filter the inputted word
    len_max = 0 #This stores the value of the maximum LCS
    word_index = 0 #This will keep track of the index for which the LCS was maximum
    for i in range(len(checker_text)): #We will go through each of the words.
        #Filter 1: If the first letter of both the words are the same, we move forward
        if checker_text[i][0] == typo_word[0][0]:
            #Finds the LCS from the two words
            lo = longest_common_subsequence(checker_text[i], typo_word[0])
            #If the new LCS is greater than the previous max,and the threshold is met, we update the values
            if len_max < lo and abs(len(checker_text[i]) - len(typo_word[0]))<= (thres * len(typo_word[0])):
                word_index = i
                len_max = lo

    return (checker_text[word_index]) #Returns the corrected word.

In [8]:
#This is the function to check if a word exists in the dictionary.
def if_exists(search_word, correct_words):
    l = filtered_words(correct_words)#We add one element to the list.
    seed_list = [3,4,5,6,7,1] #Set the seeds.
    cbf = CountingBloomFilter(0.0001,len(texts), seed_list,0) #Create an object with the given parameters
    for i in l: # Goes through the list 'l'.
        cbf.insert(i) #Insert the words inside the CBF.
    if cbf.search(search_word):
        return True #Searches for the word and if found return True
    else:
        return False #If not found, return False

In [9]:
def autocorrect1(test_word, texts, thres):
    #If the word exists, we simply return the word
    if (if_exists(test_word, texts)) == True:
        return test_word
    #If we don't find the word, we call the function corrected_word
    elif (if_exists(test_word, texts)) == False:
        return corrected_word(test_word, texts, thres)
        
    else:
        print("Error")

In [10]:
def autocorrect_1(mispell):
    thres = 0.3
    mispelled_ = filtered_words(mispell)
    corrected = []
    for i in mispelled_:
        corrected.append(autocorrect1(i, texts, thres))
    return corrected

In [11]:
#Test cell
test_line = "helo worl , thhat thesee timess "
filtered_line = filtered_words(test_line)
corrected_line = []
thres = 0.3
for i in filtered_line:
    corrected_line.append(autocorrect1(i,texts, thres))
print (corrected_line)

['hello', 'world', 'that', 'these', 'times']


### Edit Distance
The second approach will consider the edit distance between two strings. First, let's understand what edit distance encompasses. It is a measure that gives us the difference between two strings. 
Even in this case, we will not use the brute force approach i.e. find out all the strings whose edit distance is 1,2,... and terminate when the other string is achieved. This is incredibly inefficient with a time complexity of $O(2^n)$.

The most popular edit distance algorithm is the Levenshtein Distance which uses three operations i.e. insertion, deletion and substitution to find the minimum number of edits we can make to get from one string to another. We will be using dynamic programming in this case since it satisfies both the properties. 
1. Overlapping subproblems : Just like with finding LCS we can use  table to store the minimum edit distances. So, when we need to find the new edit distance, we can find the previous minimum edit distance and decide on the optimal operations.
2. Optimal substructure: We can break the problem into smaller subproblems and use recursion to find the final result.

I will show one example on how this works. Let's say that the misspelled word is 'breaken' and the correct word is 'broken'. The steps would be:
* breaken $\rightarrow$ broaken (Substituting 'e' with 'o')
* broaken $\rightarrow$ broken (Delete 'a')

But, in this implementation we will be using a slightly modified version of Levenshtein Distance called Damerau_Levenshtein Distance. The difference is that this method also considers tranposes i.e. swapping of the letter.

We will be choosing the words that are closest to one another i.e. have the smallest edit distance. We don't want to increase the edit distance to be too high since it will substantially increase the possible operations increasing the time complexity. So, for this purpose, we will be looking at edit distance of 2.

We will be using 'counter' as a data structure. We could have used Python dictionary, but there are some useful methods in 'counter'. It can tell us how many times we have seen the word in the initial dictionary. We will be using this data to get information about probabilities.

In [12]:
#This funcion fetches all the possible combinations of the given word
def possible_breaks(test_word):
    li = []
    for i in range (len(test_word)+1):
        li.append([test_word[:i], test_word[i:]]) #Appends the possible combination to li.
    return li
possible_breaks('hello')

[['', 'hello'],
 ['h', 'ello'],
 ['he', 'llo'],
 ['hel', 'lo'],
 ['hell', 'o'],
 ['hello', '']]

In [13]:
def dist_1(test_word): #Gives us a set of all possible 1 distance edits
    all_letters = 'abcdefghijklmnopqrstuvwxyz' #For adding letters to the word.
    removals = [] #List of words after removing one letter from the word
    swap = [] #for transposes
    insert = [] #For insertions
    subs = []  #For substitutions
    comb_s = possible_breaks(test_word) #Possible splits of the word
    for (i,j) in comb_s: #Run until the second element of the list is empty.
        if j:
            removals=removals+([i+j[1:]]) #Removes the letter sequentially.
    for (i,j) in comb_s: #Similar to above
        if len(j)>1: #We cannnot swap one letter so it must be more than 1.
            swap=swap+([i+j[1]+j[0]+j[2:]]) #Swap each combination.
    for (i,j) in comb_s:
        for k in all_letters: #Add each letter in all position
            insert=insert+([i+k+j])
    for (i,j) in comb_s:
        for k in all_letters:
            if j:
                subs=subs+[i+k+j[1:]] #Substitude each of the letter with the alphabets
    return set(removals+swap+subs+insert)#Gets rid of all duplicates.

In [14]:
from collections import Counter #Import the module counter
words = filtered_words(text) #Filter the text
occur = Counter(words) #Convert it into counter

In [15]:
def correct_(words):
    return {i for i in words if i in occur} #Returns the set of words that are in the initial dictionary.
correct_('the')

{'e', 'h', 't'}

In [16]:
def dist_0(test_word): #All set of words that are zero edits away from the initial word
    return {test_word} #Returns the word itself
dist_0('the')

{'the'}

In [17]:
def dist_2(test_word):#All set of words that are two edits away from the initial word
    d2 = [] 
    #One loop iterates through the list of words from dist_1 and other from the sublist of dist_1
    for i in dist_1(test_word):
        for j in dist_1(i):
            d2.append(j) #Adds all the possible edits to the list
    return set(d2)

In [18]:
def the_chosen_one(test_word):
    #Finds the correct word from the list
    c = (correct_(dist_0(test_word)) or correct_(dist_1(test_word)) or correct_(dist_2(test_word)) or [test_word])
    return max(c, key=occur.get), c

Now, let's understand how we weight our final choices using the number of times the word is seen in the dictionary text. The counter gives us the number of times each word is seen in the text, so we leverage that to get to the optimal answer. The example is shown below:

In [19]:
#This gives us the top choice that we obtained.
k = the_chosen_one('doasd')[1]
for i in k:
    print (i, 'occurs ', occur[i], 'times')

road occurs  1 times
does occurs  10 times
dash occurs  5 times
coast occurs  43 times
dead occurs  16 times
boast occurs  1 times
dogs occurs  1 times
doesn occurs  4 times
load occurs  2 times
dost occurs  1 times
board occurs  58 times


From the code given above, we can see how many times the word is seen in the initial text. So, we choose the one that occurs the most number of times. In this case, it would be 'board'.

In [20]:
#This would be the suggestion the user would get.
the_chosen_one('doasd')[0]

'board'

In [21]:
def autocorrect_2(mispell):
    mispelled_ = filtered_words(mispell)
    corrected = []
    for i in mispelled_:
        corrected.append(the_chosen_one(i)[0])
    return corrected

In [22]:
autocorrect_1('helloo worlnd, wlcome to eartyh')

['hello', 'world', 'welcome', 'to', 'earth']

In [23]:
autocorrect_2('helloo worlnd, wlcome to eartyy')

['hello', 'world', 'welcome', 'to', 'party']

In [24]:
search_word = 'hesdu'
print ("Do you mean: ", autocorrect_2(search_word))
print ('Suggestions:')
suggestions = ''
for i in (the_chosen_one(search_word)):
    print (i)

Do you mean:  ['head']
Suggestions:
head
{'heads', 'head', 'held', 'heed'}


### Analysis
#### LCS and CBFs:
This concludes our implementation of the spell checker using two distinct methods. The first method consisted of using the longest common subsequence to give the closest suggestion. This method used counting bloom filters as the data structure which can perform insertions and lookup in constant time. However, we also had to find the longest common subsequence between the misspelled word and every other word in the dictionary. If there are a total of 'n' words in the dictionary, and the length of each word is given as 'm', the worst case time complexity would be $O(nm^2)$. There are also other operations like reading the data and filtering it which takes $O(n)$ and $O(1)$ respectively.

This implementation also covers some extra filters like stabilizing the first letter and having a threshold on the difference between the length of the words. So, this improves the accuracy substantially. We can also use a shorter initial dictionary, but it is much slower than the second approach. The correct word dictionary for this approach has around 75000 words compared to the other one at 425000 words. Trying to use the other dictionary was very slow and could not improve the accuracy since no account is made for the frequency of the words.

#### Counters and Levenshtein Distance:
The data structure that we use i.e. counter is a subclass of dictionary so the time complexity is the same as dictionary i.e. $O(n)$ since it has to go through each of the inputs. Then, the levenshtein distance takes $O(mn)$ to find the minimum edit distance, where 'm' and 'n' are the length of the strings. However, we need to iterate it over the entire counter. So, if we consider 'x' to be the length of the counter, the time complexity would be $O(mnx)$.

The biggest pro of this method is the ability to get suggestions rather than only the final word. Since, we are weighting the words based on their frequency of occurrence, the even if the final answer is wrong, other suggestions should be very close. Another pro is the use of counters from which we could further find the probabilities of each words, and even phrases. This is not possible with the first approach. The time complexity is also better than the first approach, and the accuracy is also much greater.